In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [3]:
path = "../training/conf/experiment/cnn_htr_char_lines.yaml"

In [7]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [8]:
cfg

{'defaults': [{'override /mapping': None}, {'override /criterion': None}, {'override /datamodule': None}, {'override /network': None}, {'override /model': None}, {'override /lr_schedulers': None}, {'override /optimizers': None}], 'ignore_index': 3, 'num_classes': 58, 'max_output_len': 89, 'criterion': {'_target_': 'torch.nn.CrossEntropyLoss', 'ignore_index': 3}, 'mapping': {'_target_': 'text_recognizer.data.emnist_mapping.EmnistMapping'}, 'callbacks': {'stochastic_weight_averaging': {'_target_': 'pytorch_lightning.callbacks.StochasticWeightAveraging', 'swa_epoch_start': 0.8, 'swa_lrs': 0.05, 'annealing_epochs': 10, 'annealing_strategy': 'cos', 'device': None}}, 'optimizers': {'madgrad': {'_target_': 'madgrad.MADGRAD', 'lr': 0.0003, 'momentum': 0.9, 'weight_decay': 0, 'eps': 1e-06, 'parameters': 'network'}}, 'lr_schedulers': {'network': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau', 'mode': 'min', 'factor': 0.5, 'patience': 10, 'threshold': 0.0001, 'threshold_mode': 'rel', '

In [9]:
net = instantiate(cfg.network)

In [10]:
from torchinfo import summary

In [16]:
summary(net, list(map(lambda x: list(x), cfg.summary)), device="cpu", depth=1)

tensor([[[[-0.1375,  0.1336, -0.9556,  ...,  1.3900, -0.1311,  0.0424],
          [ 0.3474,  0.3920, -0.7718,  ...,  1.3900, -0.1311,  0.0424],
          [ 0.5129,  0.5296, -0.5113,  ...,  1.3900, -0.1311,  0.0424],
          ...,
          [-0.4101, -0.5381,  0.8383,  ...,  1.3900, -0.1311,  0.0424],
          [-0.4902, -0.4792,  0.9942,  ...,  1.3900, -0.1311,  0.0424],
          [-0.1197, -0.2727,  1.0515,  ...,  1.3900, -0.1311,  0.0424]],

         [[-0.5087, -0.9362,  0.4579,  ..., -0.9192,  0.5128, -0.4577],
          [-0.9311, -0.7565,  0.2538,  ..., -0.9192,  0.5128, -0.4577],
          [-0.4974, -0.3438,  0.0245,  ..., -0.9192,  0.5128, -0.4577],
          ...,
          [ 0.9153,  0.2433, -0.7304,  ..., -0.9192,  0.5128, -0.4577],
          [ 0.3510, -0.2775, -0.7340,  ..., -0.9192,  0.5128, -0.4577],
          [-0.5360, -0.7128, -0.6648,  ..., -0.9192,  0.5128, -0.4577]],

         [[-0.3042,  0.4230, -0.1477,  ...,  0.5753,  0.2627, -0.9272],
          [-0.6520,  0.1020, -

Layer (type:depth-idx)                             Output Shape              Param #
ConvTransformer                                    --                        --
├─EfficientNet: 1-1                                [1, 1280, 1, 32]          7,142,272
├─Sequential: 1-2                                  [1, 128, 32]              163,968
├─Embedding: 1-3                                   [1, 89, 128]              7,424
├─PositionalEncoding: 1-4                          [1, 89, 128]              --
├─Decoder: 1-5                                     [1, 89, 128]              13,176,969
├─Linear: 1-6                                      [1, 89, 58]               7,482
Total params: 20,498,115
Trainable params: 20,498,115
Non-trainable params: 0
Total mult-adds (M): 714.86
Input size (MB): 0.23
Forward/backward pass size (MB): 184.29
Params size (MB): 81.99
Estimated Total Size (MB): 266.51